In [411]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [412]:
import numpy as np
import torch
import torch.nn as nn
import torchvision


from model import FreeFormImageInpaint
from preprocess import get_images, get_mask
from torchsummary import summary
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [413]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [414]:
n_epochs = 1
alpha = 0.02
learning_rate = 0.001
batch_size = 15

In [415]:
image_loader = get_images(batch_size)
image_size = (256, 256)
square_size = 30
binary_mask = get_mask(image_size, square_size)
# reshape binary mask to add batch_size dimension
binary_mask = binary_mask.unsqueeze(0)
binary_mask = binary_mask.expand(batch_size, -1, -1)
binary_mask = binary_mask.to(device)

print(binary_mask.shape)

Files already downloaded and verified
torch.Size([10, 256, 256])


In [416]:
from torchsummary import summary
inpaint = FreeFormImageInpaint(in_channels=4).to(device)
summary(inpaint, [(3, 256, 256), (256, 256)])

Layer (type:depth-idx)                   Output Shape              Param #
├─Sequential: 1-1                        [-1, 3, 256, 256]         --
|    └─GatedConv: 2-1                    [-1, 32, 256, 256]        --
|    |    └─Conv2d: 3-1                  [-1, 32, 256, 256]        3,232
|    |    └─Conv2d: 3-2                  [-1, 32, 256, 256]        3,232
|    |    └─Sigmoid: 3-3                 [-1, 32, 256, 256]        --
|    |    └─LeakyReLU: 3-4               [-1, 32, 256, 256]        --
|    └─GatedConv: 2                      []                        --
|    |    └─LeakyReLU: 3-5               [-1, 32, 256, 256]        --
|    └─GatedConv: 2                      []                        --
|    |    └─LeakyReLU: 3-6               [-1, 32, 256, 256]        --
|    └─GatedConv: 2                      []                        --
|    |    └─LeakyReLU: 3-7               [-1, 32, 256, 256]        --
|    └─GatedConv: 2                      []                        --
|    |   

Layer (type:depth-idx)                   Output Shape              Param #
├─Sequential: 1-1                        [-1, 3, 256, 256]         --
|    └─GatedConv: 2-1                    [-1, 32, 256, 256]        --
|    |    └─Conv2d: 3-1                  [-1, 32, 256, 256]        3,232
|    |    └─Conv2d: 3-2                  [-1, 32, 256, 256]        3,232
|    |    └─Sigmoid: 3-3                 [-1, 32, 256, 256]        --
|    |    └─LeakyReLU: 3-4               [-1, 32, 256, 256]        --
|    └─GatedConv: 2                      []                        --
|    |    └─LeakyReLU: 3-5               [-1, 32, 256, 256]        --
|    └─GatedConv: 2                      []                        --
|    |    └─LeakyReLU: 3-6               [-1, 32, 256, 256]        --
|    └─GatedConv: 2                      []                        --
|    |    └─LeakyReLU: 3-7               [-1, 32, 256, 256]        --
|    └─GatedConv: 2                      []                        --
|    |   

In [417]:
inpaint = FreeFormImageInpaint(in_channels=4).to(device)
optimizer = torch.optim.Adam(inpaint.parameters(), lr=learning_rate)

In [418]:
len(image_loader)

3061

In [419]:
for epoch in range(n_epochs):
    train_loss = 0
    loss = 0
    for batch_idx, (data, _) in enumerate(image_loader):
        print(batch_idx)
        if data.shape[0] != batch_size:
            continue
        data = data.to(device)
        binary_mask = binary_mask.to(device)
        # clear the gradients of all optimized variables
        optimizer.zero_grad()
        # model forward
        x_hat = inpaint(data, binary_mask)
        #print("shape of output:", x_hat.shape)
        # compute the loss
        loss = inpaint.loss_function(x_hat, data, binary_mask, alpha)
        # model backward
        loss.backward()
        # update the model paramters
        optimizer.step()
        # update running training loss
        train_loss += loss
    train_loss = train_loss/len(image_loader)
    print('Epoch: {} \tTraining Loss: {:.6f}'.format(epoch, train_loss))

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

KeyboardInterrupt: 